### IMDB 영화평 감성분석
- Pipeline
- TfidVectorizer + LogisticRegression

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- 텍스트 전처리

In [ ]:
# <br /> 공백으로 변환
df.review = df.review.str.replace('<br />', ' ')

# 구두점, 숫자 제거 - 영문자가 아닌 글자는 공백으로 변환
df.review = df.review.str.replace('[^A-Za-z]', ' ').str.strip()
df.review[0][:1000]

'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- Train/Test dataset 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.review, df.sentiment, stratify=df.sentiment, random_state=2022)

- Pipeline: TfidVectorizer + LogisticRegression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
tvect = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
lrc = LogisticRegression(random_state=2022)
pipeline = Pipeline([('TVECT', tvect), ('LR', lrc)])

In [ ]:
# 학습
%time pipeline.fit(X_train, y_train)

CPU times: user 32.9 s, sys: 13.9 s, total: 46.9 s
Wall time: 32.2 s


Pipeline(steps=[('TVECT',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('LR', LogisticRegression(random_state=2022))])

In [ ]:
pipeline.score(X_test, y_test)

0.87472

- 최적 파라미터 찾기

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'TVECT__max_df': [100, 500], 'LR__C': [1, 10]}

In [ ]:
grid_pipe = GridSearchCV(pipeline, params, scoring='accuracy', cv=3, n_jobs=-1)
%time grid_pipe.fit(X_train, y_train)

CPU times: user 41.8 s, sys: 19.6 s, total: 1min 1s
Wall time: 3min 1s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('TVECT',
                                        TfidfVectorizer(ngram_range=(1, 2),
                                                        stop_words='english')),
                                       ('LR',
                                        LogisticRegression(random_state=2022))]),
             n_jobs=-1,
             param_grid={'LR__C': [1, 10], 'TVECT__max_df': [100, 500]},
             scoring='accuracy')

In [ ]:
grid_pipe.best_params_

{'LR__C': 10, 'TVECT__max_df': 500}

In [ ]:
grid_pipe.best_estimator_.score(X_test, y_test)

0.87552

- 모델 저장하고 불러오기

In [ ]:
import joblib
joblib.dump(grid_pipe.best_estimator_, 'imdb_tvect_lr.pkl')

['imdb_tvect_lr.pkl']

In [ ]:
best_pipe = joblib.load('imdb_tvect_lr.pkl')

- 실제 데이터 적용

In [ ]:
review = '''
Got to see the early ish screening considered date night on the regal app that was great time!
One of the funniest things set up in the is experience was showing a short trailer of the movie 6 times that's never been a thing
and there was feature before the movie started mostly talking about the red dress being a character itself that was silly.
Didn't expect as much comedy as there was and highly entertaining action even an excellent soundtrack!
I don't know about others but I loved the movie.
'''

In [ ]:
# 텍스트 전처리
import re
review = re.sub('[^A-Za-z]', ' ', review).strip()

In [ ]:
best_pipe.predict([review])

array([1])